<a href="https://colab.research.google.com/github/mephi82/grad_data_analysis/blob/main/recurrentnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

자동 음악 생성
https://www.analyticsvidhya.com/blog/2020/01/how-to-perform-automatic-music-generation/

In [ ]:
from tensorflow.keras import datasets, layers, utils, models, optimizers
import matplotlib.pyplot as plt
import numpy as np

문자 생성 예시 : https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [ ]:
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()[1:]

create mapping of unique chars to integers

In [ ]:
chars = sorted(list(set(raw_text[1:])))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(chars)

['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163780
Total Vocab:  58


prepare the dataset of input to output pairs encoded as integers

In [ ]:
seq_length = 20
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163760


In [ ]:
X = utils.to_categorical(dataX)
y = np.reshape(dataY, (n_patterns, ))
(X.shape, y.shape)

((163760, 20, 58), (163760,))

Define RNN models

In [ ]:
def simplernn(seq_length, n_vocab):
  model = models.Sequential()
  model.add(layers.SimpleRNN(128, return_sequences=False, unroll= True, input_shape=(seq_length, n_vocab)))
  model.add(layers.Dense(n_vocab, activation='softmax'))           
  return(model)

def lstm(seq_length, n_vocab):
  model = models.Sequential()           
  model.add(layers.LSTM(256, input_shape=(X.shape[1], X.shape[2])))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(n_vocab, activation='softmax'))
  return(model)  

model = lstm(seq_length, n_vocab)
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               322560    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 58)                14906     
Total params: 337,466
Trainable params: 337,466
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
hist = model.fit(X, y, epochs=10, batch_size=128)

Epoch 1/10
1280/1280 [==============================] - 19s 13ms/step - loss: 0.8940 - accuracy: 0.7202
Epoch 2/10
1280/1280 [==============================] - 16s 12ms/step - loss: 0.8638 - accuracy: 0.7283
Epoch 3/10
1280/1280 [==============================] - 16s 12ms/step - loss: 0.8462 - accuracy: 0.7325
Epoch 4/10
1280/1280 [==============================] - 16s 12ms/step - loss: 0.8239 - accuracy: 0.7391
Epoch 5/10
1280/1280 [==============================] - 15s 12ms/step - loss: 0.8059 - accuracy: 0.7442
Epoch 6/10
1280/1280 [==============================] - 15s 12ms/step - loss: 0.7914 - accuracy: 0.7484
Epoch 7/10
1280/1280 [==============================] - 15s 12ms/step - loss: 0.7730 - accuracy: 0.7532
Epoch 8/10
1280/1280 [==============================] - 15s 12ms/step - loss: 0.7579 - accuracy: 0.7578
Epoch 9/10
1280/1280 [==============================] - 15s 12ms/step - loss: 0.7476 - accuracy: 0.7609
Epoch 10/10
1280/1280 [==============================] - 14s 11m

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
def generate_text(model, pattern):
    text = ''
    for i in range(200):
        x = utils.to_categorical(pattern, num_classes= n_vocab)
        x = np.expand_dims(x, axis = 0)
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = int_to_char[index]
        print(result, end='')
        seq_in = [int_to_char[value] for value in pattern]
        text += result
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    print("\nDone.")
    return(text)

generate_text(model, pattern)

Seed:
"  on;
then, when you' "
ve no remeaber dound in it again, and the most bust tome this time, when she began again: and i'm sure i mean would be a little baty cain of eneraling about permans, and alice was soing
to bees, and t
Done.


"ve no remeaber dound in it again, and the most bust tome this time, when she began again: and i'm sure i mean would be a little baty cain of eneraling about permans, and alice was soing\nto bees, and t"